In [39]:
import warnings

from pipeline.DBHandler import DBHandler

# chat imports
import google.generativeai as genai

import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

from pipeline.Chatbot import Chatbot


In [40]:
# class Chatbot:
# 	# Todo: Support multiple LLMs
# 	def __init__(self, user_id: str, llm_model_name: str = 'gemini-1.5-flash',
# 				 embedding_model_name: str = 'models/text-embedding-004'):
# 		"""
# 		Initializes the Chat object
# 		Args:
# 			user_id (str): The user id, used to connect to the correct collections in the database
# 			llm_model_name (str): The name of the model to use
# 			embedding_model_name (str): the name of the model to use for the embedding, either 'models/text-embedding-004' or 'models/embedding-001'
# 		Raises:
# 			ValueError: If the db_handler is not an instance of DBHandler
# 			ValueError: If the model name is not supported
# 			RuntimeError: If there was an error initializing the model
# 		"""
# 		self.db_handler = DBHandler(user_id, os.getenv('MONGODB_CONNECTION_STRING'))
# 
# 		# Initialize the LLM
# 		possible_models = []
# 		for option in genai.list_models():
# 			if 'generateContent' in option.supported_generation_methods:
# 				possible_models.append(option.name)
# 		if not llm_model_name and isinstance(llm_model_name, str) and f'models/{llm_model_name}' not in possible_models:
# 			raise ValueError(f'Model name must be one of the following: {possible_models}')
# 		try:
# 			self.llm = genai.GenerativeModel(llm_model_name)
# 		except Exception as e:
# 			raise RuntimeError(f'Error initializing model: {e}')
# 
# 		# Validate the embedding model name
# 		if not embedding_model_name or embedding_model_name not in ['models/text-embedding-004',
# 																	'models/embedding-001']:
# 			raise ValueError('Invalid embedding model name')
# 		self.embedding_model_name = embedding_model_name
# 
# 	def interact(self, query: str) -> str:
# 		"""
# 		Interacts with the model
# 		Args:
# 			query (str): The query to interact with
# 		Returns:
# 			str: The response from the model
# 		"""
# 		try:
# 			response = self.llm.generate_content(query)
# 			return response.text.strip('\n').strip(' ')
# 		except Exception as e:
# 			raise RuntimeError(f'Error interacting with model: {e}')
# 
# 	def rephrase_question(self, query: str) -> str:
# 		"""
# 		Rephrases the last user's question as a standalone question
# 		Returns:
# 			str: The rephrased question
# 		Raises:
# 			ValueError: If the query is not a non-empty string
# 		"""
# 		if not query or not isinstance(query, str):
# 			raise ValueError('Query must be a non-empty string')
# 
# 		chat_history = self.db_handler.get_history()
# 		chat_history.append(f'user: {query}')
# 
# 		# Only use the last 5 messages in the chat history to keep the context relevant
# 		prompt = f"""
# 			   Your job is to rephrase the last user's question as a standalone question that can be understood without the context that is provided in the chat history.
# 			   If the user's question is already standalone, just return it as it is.
# 			   Chat history: {chat_history[-5:]}
# 			   """
# 		response = self.interact(prompt)
# 		return response
# 
# 	def google_embedding(self, text: str) -> list:
# 		"""
# 		Embeds the text using the embedding model
# 		Args:
# 			text (str): the text to embed
# 		Returns:
# 			embedding (list): the embedding vector of the text
# 		Raises:
# 			Exception: if there is an error in embedding the text
# 		"""
# 		try:
# 			embedding = genai.embed_content(model=self.embedding_model_name, content=text,
# 											task_type='retrieval_document')
# 		except Exception as e:
# 			raise Exception(f'Error in embedding the text: {e}')
# 
# 		return embedding['embedding']
# 
# 	def answer_question(self, query: str) -> str:
# 		"""
# 		Run a user's question through the RAG pipeline and return the answer
# 		Args:
# 			query (str): The user's question
# 		Returns:
# 			str: The answer to the user's question
# 		"""
# 		rephrased_query = self.rephrase_question(query)
# 
# 		query_vector = self.google_embedding(rephrased_query)
# 		relevant_chunks = self.db_handler.search(query_vector)
# 		if relevant_chunks:
# 			# Todo: Improve the logic for selecting the context
# 			context = '\n\n\n'.join([chunk['text'] for chunk in relevant_chunks if chunk['score'] > 0.3])
# 		else:
# 			warnings.warn('No relevant context found in the database')
# 			context = ''
# 
# 		rag_prompt = f"""
# 				   Your job is to answer the user's question based only on the provided context.
# 				   If you are unable to answer the question based on the context provided, you can ask for more information.
# 				   Even if you don't have the direct answer, provide the best response you can.
# 				   User's question: {rephrased_query}
# 				   Context: {context}
# 				   """
# 
# 		response = self.interact(rag_prompt)
# 
# 		messages_to_append = [
# 			{
# 				'role': 'user',
# 				'content': rephrased_query
# 			},
# 			{
# 				'role': 'bot',
# 				'content': response
# 			}
# 		]
# 
# 		self.db_handler.update('history', messages_to_append)
# 
# 		return response



In [41]:
chat = Chatbot(user_id='maccabi')

In [8]:
response = chat.answer_question('Can you explain the process of terminating a pregnancy?')
print(response)

The text provided focuses on the practicalities of termination of pregnancy in Israel, specifically the financial aspects and procedures involved. It doesn't elaborate on the medical details of the process itself. 

To answer your question about the process, you'll need to consult reliable sources such as medical websites or textbooks.


In [111]:
response = chat.answer_question('What are the conditions that must be met so maccabi will cover the cost of the process?')
print(response)

In [112]:
response = chat.answer_question('What is the biggest planet in the solar system?')
print(response)

### Test tzizo questions

In [19]:
response = chat.answer_question('What is the quarterly cost to visit a specialist?')
print(response)

The quarterly cost to visit a specialist is 36₪.


In [20]:
response = chat.answer_question('And how is it charged?')
print(response)

The context doesn't explain how the 36₪ quarterly cost for visiting a specialist is charged.  It only mentions that there is a 36₪ quarterly deductible. It doesn't say if the cost is charged once per quarter or if it is charged for every visit.


In [21]:
response = chat.answer_question('Can I come to a maccabi health institutions with my dog?')
print(response)

The text you provided is about the rules for bringing service animals into Maccabi health institutions. You are asking if you can bring your dog.  Unfortunately, the text does not state whether a pet dog is allowed.  The text focuses on service animals, which have specific requirements.


In [36]:
response = chat.answer_question('Can I come to a health institutions with a service animal?')
print(response)

It depends on what kind of health institution you are visiting. Service animals are generally allowed in most health institutions, but there are some exceptions.  

For example, service animals are not allowed in places where surgery, catheters, other invasive operations, and dialysis are performed. They are also not allowed in oncological clinics, except in waiting places outside the treatment rooms.  Service animals are also not allowed in outpatient departments or clinics designed to treat people with immunodeficiency or whose patients are usually immunodeficient, except in waiting places outside the treatment rooms.  

Would you like to provide the name or type of health institution you are interested in?


In [37]:
response = chat.answer_question('A Family doctor')
print(response)

Yes, you can bring a service animal to a family doctor's office, as long as it is a health institution committed to allowing entry with a service animal.


In [42]:
response = chat.answer_question('Is there a treatment of eating disorders in Holon?')
print(response)

I'm sorry, but the provided context doesn't list any clinics offering eating disorder treatment in Holon.
